In [107]:
import numpy as np
import timeit
import dolfin as df

Create a simple rectangular mesh

In [78]:
mesh = df.RectangleMesh(-2, -2, 2, 2, 2, 2)

In [12]:
df.plot(mesh)

<dolfin.cpp.io.VTKPlotter; proxy of <Swig Object of type 'std::shared_ptr< dolfin::VTKPlotter > *' at 0x7f5365cc3fc0> >

In [79]:
mesh.coordinates()

array([[-2., -2.],
       [ 0., -2.],
       [ 2., -2.],
       [-2.,  0.],
       [ 0.,  0.],
       [ 2.,  0.],
       [-2.,  2.],
       [ 0.,  2.],
       [ 2.,  2.]])

Create an scalar function space

In [80]:
V_space = df.FunctionSpace(mesh, "CG", 1)

DEBUG:UFL:No integrals left after transformation, returning empty form.
DEBUG:FFC:Reusing form from cache.


Now we create a Mixed Function Space with 3 scalar spaces

In [82]:
MixedSpace = df.MixedFunctionSpace([V_space, V_space, V_space])

DEBUG:UFL:No integrals left after transformation, returning empty form.
DEBUG:FFC:Reusing form from cache.


We can create a dolfin function of the mixed space, which will have a function for every function space

In [83]:
Mixed_df_Function = df.Function(MixedSpace)

We can print all the scalar field values, which are 27 in total (9 for every function space inside the Mixed element)

In [84]:
Mixed_df_Function.vector().array()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

We can get every dolfin function splitting the Mixed space

In [85]:
a, b, c = Mixed_df_Function.split()

In [86]:
a.this

<Swig Object of type 'std::shared_ptr< dolfin::Function > *' at 0x7f535dec0270>

However, it seems that interpolating one of the splitted obejects, does not work

In [ ]:
# a.interpolate(df.Expression("x[0] + x[1]"))

We can solve this issue interpolating a single dolfin function, and then assign its value to any of the sub-functions of the Mixed dolfin function element (see http://fenicsproject.org/qa/2157/projection-on-subspace-of-mixed-function-space):

In [87]:
s = df.Function(V_space)
s.interpolate(df.Expression("x[0] + x[1]"))

# (With uncached dofmaps) We assign the values
# to the 2th dolfin function inside the Mixed element
df.assign(Mixed_df_Function.sub(2), s)

We can see that the elements in the second space are updated:

In [88]:
Mixed_df_Function.vector().array()

array([ 0.,  0., -4.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0., -2.,  0.,
        0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  4.])

Furthermore, we can do this in the opposite way, i.e. assign the values from the Mixed element to the single function (thus we can retrieve the data from the Mixed space)

In [93]:
# Start the space again
MixedSpace = df.MixedFunctionSpace([V_space, V_space, V_space])
Mixed_df_Function = df.Function(MixedSpace)

DEBUG:UFL:No integrals left after transformation, returning empty form.
DEBUG:FFC:Reusing form from cache.


In [94]:
# Print the 's' function
s.vector().array()

array([-4., -2.,  0., -2.,  0.,  2.,  0.,  2.,  4.])

In [96]:
# Assign the values from the Mixed element to the
# single function and print. We will see the elements
# of 's' turning to zeros
df.assign(s, Mixed_df_Function.sub(2))

s.vector().array()

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

A different approach is to use the indexes from the single vector space and assign the values with these values

In [103]:
# Restart the elements
MixedSpace = df.MixedFunctionSpace([V_space, V_space, V_space])
Mixed_df_Function = df.Function(MixedSpace)

s = df.Function(V_space)
s.interpolate(df.Expression("x[0] + x[1]"))

DEBUG:UFL:No integrals left after transformation, returning empty form.
DEBUG:FFC:Reusing form from cache.


In [104]:
# (with cached dofmaps) Save the dofs of the single function
# and then assign the values from 's' to the the Mixed space
# (probably it is faster) We will assign values to the 0th function
assigner = df.FunctionAssigner(MixedSpace.sub(0), V_space)
assigner.assign(Mixed_df_Function.sub(0), s)

In [105]:
Mixed_df_Function.vector().array()

array([-4.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0., -2.,  0.,  0.,  0.,
        0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  4.,  0.,  0.])

We can time the assignment of both methods:

In [114]:
%timeit -n 10000 df.assign(s, Mixed_df_Function.sub(0))

10000 loops, best of 3: 1.09 ms per loop


In [115]:
%timeit -n 10000 assigner.assign(Mixed_df_Function.sub(0), s)

10000 loops, best of 3: 1.08 ms per loop
